In [2]:
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt

In [4]:
seq_model = tf.keras.Sequential([
    tf.keras.Input(shape=(None, 40, 40, 1)), 
    tf.keras.layers.ConvLSTM2D(filters=40, kernel_size=(3, 3), padding='same', return_sequences=True),
    
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ConvLSTM2D(filters=40, kernel_size=(3, 3), padding='same', return_sequences=True),
    
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ConvLSTM2D(filters=40, kernel_size=(3, 3), padding='same', return_sequences=True),
    
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ConvLSTM2D(filters=40, kernel_size=(3, 3), padding='same', return_sequences=True),
    
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv3D(filters=1, kernel_size=(3, 3, 3), padding='same', activation='sigmoid')
])

seq_model.compile(loss='binary_crossentropy', optimizer='adadelta')

In [5]:
def generate_movies(n_samples=1200, n_frames=15):
    row = 80 
    col = 80 
    
    noisy_movies = np.zeros((n_samples, n_frames, row, col, 1), dtype=np.float) 
    
    shifted_movies = np.zeros((n_samples, n_frames, row, col, 1), dtype=np.float)
    
    for i in range(n_samples): 
        
        n = np.random.randint(3, 8)
        
        for j in range(n):
            
            x_start = np.random.randint(20, 60)
            y_start = np.random.randint(20, 60)
            
            direction_x = np.random.randint(0, 3) - 1
            direction_y = np.random.randint(0, 3) - 1
            
            w = np.random.randint(2, 4) 
            
            
            for t in range(n_frames):
                x_shift = x_start + direction_x + t 
                y_shift = y_start + direction_y + t 
                
                noisy_movies[i, t, x_shift - w : x_shift + w, y_shift - w : y_shift + w, 0] += 1
                
                if np.random.randint(0, 2): 
                    
                    noise_f = (-1) ** np.random.randint(0, 2) 
                    
                    noisy_movies[i, t, x_shift - w - 1 : x_shift + w + 1, y_shift - w - 1 : y_shift + w + 1, 0] += (noise_f * 0.1) 
                    
            
            # 정답 데이터
            x_shift = x_start + direction_x * (t + 1) 
            y_shift = y_start + direction_y * (t + 1) 
            shifted_movies[i, t, x_shift - w : x_shift + w, y_shift - w : y_shift + w, 0] += 1
            
            
    noisy_movies = noisy_movies[::, ::, 20:60, 20:60, ::]
    shifted_movies = shifted_movies[::, ::, 20:60, 20:60, ::]
                
                
    noisy_movies[noisy_movies >= 1] = 1 
    shifted_movies[shifted_movies >= 1] = 1
    
    return noisy_movies, shifted_movies
        
        
    
    

In [6]:
epochs = 1

noisy_movies, shifted_movies = generate_movies(n_samples=1200) 

seq_model.fit(noisy_movies[:1000], shifted_movies[:1000], batch_size=10, epochs=epochs, verbose=2, validation_split=0.1)

KeyboardInterrupt: 